# Mandelbrot Set


Explore the dynamic nature of the Mandelbrot and Julia sets.

### References

 <br />
 

  https://en.wikipedia.org/wiki/Mandelbrot_set
 
 <br />
 

  https://en.wikipedia.org/wiki/Julia_set
 
 <br />
 

[Karlsims.com](https://www.karlsims.com/julia.html) provides an excellent explanation of the nature of the Julia sets.
   
Below is a video from the above website which visualizes the process by which a Julia set image is generated.
 
 <br />
 

The [Mandelbrotbrower website](http://www.cuug.ab.ca/dewara/mandelbrot/Mandelbrowser.html)
provides a list of interesting points along the Mandelbrot set.

These points can be entered into the generator below to reproduce similar images.





In [11]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/R1gpm7WsNhg" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')


C:\Users\ras42\anaconda3\lib\site-packages\IPython\core\display.py:717: UserWarning:

Consider using IPython.display.IFrame instead



In [1]:
%matplotlib inline  
import numpy as np
from numpy import sqrt, log2, arctan2, sin, cos

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import style
from matplotlib.animation import FuncAnimation as FA
from time import time

from numba import jit, njit

import IPython
from IPython import display
# from IPython.display import display as disp
import ipywidgets as widgets

import plotly.graph_objects as go
import plotly.express as px



# plt.rcParams["figure.figsize"]=(20,20) #sets the default image size
# plt.rcParams["figure.dpi"]=(300)

#### Julia and Mandelbrot generator functions

In [2]:
@jit(forceobj=True)
def Julia(cx, cy, center_x=0, center_y=0, frame_radius=2, max_iter=100, resolution=500):
    C = complex(cx, cy)
    R = (1 + sqrt(1 + 4*sqrt(cx*cx + cy*cy)))/2 #escape radius
    
    xmin = center_x - frame_radius
    xmax = center_x + frame_radius
    ymin = center_y - frame_radius
    ymax = center_y + frame_radius
    # NxN matrix populated with zeros
    z_arr = np.zeros([resolution,resolution], dtype=np.float64, order='C')
    
    x = np.linspace(xmin, xmax, resolution)
    y = np.linspace(ymin, ymax, resolution)
   
    for _x in range(len(x)):
        for _y in range(len(y)):
            i = 0
            z = complex(x[_x],y[_y])
            while abs(z) < R and i < max_iter:
                z = z*z + C
                i +=1
                
            if abs(z) >= R:
                for j in range(2):
                    z = z*z + C
                    i +=1
                if abs(z) > 0:
                    i -= log2(log2(abs(z)))
            
            if i > 0:
                z_arr[_y,_x] = i
            else:
                z_arr[_y,_x] = 0
            
    return z_arr

@jit(forceobj=True)
def Mandelbrot(center_x, center_y, frame_radius=2, max_iter=100, resolution=500 , escape_radius = 2):

    R = escape_radius

    xmin = center_x - frame_radius
    xmax = center_x + frame_radius
    ymin = center_y - frame_radius
    ymax = center_y + frame_radius
    # NxN matrix populated with zeros
    z_arr = np.zeros([resolution,resolution], dtype=np.float64, order='C')
    
    x = np.linspace(xmin, xmax, resolution)
    y = np.linspace(ymin, ymax, resolution)

    
    for _x in range(len(x)):
        for _y in range(len(y)):
            i = 0
            z = complex(0,0)
            C = complex(x[_x],y[_y])
            while abs(z) <= R and i <= max_iter:
                z = z*z + C
                i +=1
                
            if abs(z) > R:
                for j in range(2):
                    z = z*z + C
                    i +=1

            z_arr[_y,_x] = i
            
    return z_arr
                

#### Colormaps

In [3]:
color_scales=['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg',
             'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn', 'puor',
             'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy',
             'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar', 'spectral',
             'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose',
             'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'twilight',
             'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd']

color_scales_r=[]
for _r in range(len(color_scales)):
    color_scales_r.append(color_scales[_r]+'_r')
    
all_colors = []
for c in range(len(color_scales)):
    all_colors.append(color_scales[c])
    all_colors.append(color_scales_r[c])

# print(all_colors)

#### Widgets

In [4]:
items_layout = widgets.Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='50%'
)


julia_cx = widgets.FloatText(
    value=-0.79,
    step=1/1000,
    description='Julia Re(C):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

julia_cy = widgets.FloatText(
    value=0.15,
    step=1/1000,
    description='Julia Im(C):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

resolution = widgets.IntText(
    value=600,
    min=100,
    max=10000,
    step=50,
    description='Resolution:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

max_iterations = widgets.IntText(
    value=150,
    min=1,
    max=10000,
    step=50,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

radius_input = widgets.FloatText(
    value=2,
    description='Frame Radius:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.3f',
)

frame_center_x = widgets.FloatText(
    value=0.0,
    step=1/1000,
    description='X Center:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.3f',
)

frame_center_y = widgets.FloatText(
    value=0.0,
    step=1/1000,
    description='Y Center:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.3f',
)

colors = widgets.Dropdown(
    options =all_colors,
    value ='icefire_r',
    description='Colormap',
    disabled=False
)

#Contrast Rescaling
contrast = widgets.RadioButtons(
    options=['minmax', 'infer', None ],
    value=None, # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Contrast:',
    disabled=False,
    orientation='horizontal'
)

compression = widgets.RadioButtons(
    options=['png', 'jpg'],
    value='png', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Compression:',
    disabled=False,
    orientation='horizontal'
)

plot_choice = widgets.RadioButtons(
    options=['Julia', 'Mandelbrot'],
    value='Julia',
    layout={'width': 'max-content'}, # If the items' names are long
    description='Plot Type:',
    disabled=False,
    orientation='horizontal'
)

plot_button = widgets.Button(description="Plot Image")
clear_button = widgets.Button(description="Clear Images")
buttons = widgets.HBox([plot_button,clear_button])
frame_radius = widgets.HBox([frame_center_x, frame_center_y, radius_input])
julia_cxcy = widgets.HBox([julia_cx, julia_cy])
reso_iter = widgets.HBox([plot_choice, resolution, max_iterations])
plot_color_con = widgets.HBox([colors, contrast, compression])
output = widgets.Output()


#### Widget Buttons

In [5]:

def on_button_clicked1(b):
    with output:
        
        _x = frame_center_x.value
        _y = frame_center_y.value
        radius = radius_input.value
        max_iter = max_iterations.value
        real = 'Real Numbers'
        imag = 'Imaginary Number'

        xmin = _x - radius
        xmax = _x + radius
        ymin = _y - radius
        ymax = _y + radius
        
        x=np.linspace(xmin, xmax, resolution.value)
        y=np.linspace(ymin, ymax, resolution.value)
        
        t = time()
        
        if plot_choice.value=='Julia':
            
            px.Range(label='Julia set')
            cx = julia_cx.value
            cy = julia_cy.value
            
            title = 'Julia Set: z^2 +(' + str(cx) + ')+('+ str(cy) +')i '
            print(title)
            
            z = Julia(
                cx=julia_cx.value, 
                cy=julia_cy.value, 
                center_x=_x, 
                center_y=_y, 
                frame_radius=radius, 
                max_iter=max_iterations.value,
                resolution=resolution.value
            )
    
        elif plot_choice.value=='Mandelbrot':
            
            px.Range(label='Mandelbrot set')
            
            title = 'Mandelbrot Set: z^2 + x + iy'
            print(title)
            
            z = Mandelbrot(
                center_x=_x, 
                center_y=_y, 
                frame_radius=radius, 
                max_iter=max_iterations.value,
                resolution=resolution.value
            )
        elapsed = time() - t
        
        info = 'Bounds: x-axis:[' + str(xmin) + ', ' + str(xmax)+'], y-axis:[' + str(ymin) + ', ' + str(ymax) + ']'
        print(info)
        print('Center: (' + str(_x) + ', ' + str(_y) + '), Radius:', radius)
        print('Iterations: ' + str(max_iter) + ', Colormap:', colors.value)
        
        
        fig = px.imshow(
            z,
            x=x,
            y=y,
            width=resolution.value,
            height=resolution.value,
            origin='lower',
            color_continuous_scale=colors.value,
            contrast_rescaling=contrast.value,
            labels={'x':real,'y':imag},
            binary_format=compression.value
        )
        
        print('Time:', elapsed, 'seconds')
        fig.show()

def on_button_clicked2(b):
    with output:
        IPython.display.clear_output(wait=False)



#### Plot Generator

In [6]:
IPython.display.display(frame_radius, julia_cxcy, reso_iter, plot_color_con, buttons, output)

plot_button.on_click(on_button_clicked1)
clear_button.on_click(on_button_clicked2)

Output()